In [1]:
working_directory = '/mnt/DataRAID/melismail/PDAC'
import os
os.chdir(working_directory)
from pickle_utils import write_pickle, read_pickle

import pandas as pd
import numpy as np
import sys, cv2

from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [2]:
base_path = '/mnt/DataRAID/melismail/PDAC/data'
preprocessing_path ='Preprocessing_images'
filtered_path = 'filtered'
dim = 224
dim_path = f"{dim}x{dim}"
features_path = 'Features_extraction'
model_path = 'VGG-16'

In [3]:
filtered_img_dict = read_pickle(os.path.join(base_path, preprocessing_path, filtered_path, dim_path, f"{filtered_path}_{dim_path}_img_dict.pkl"))

In [4]:
def image_feature(img, model):
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape(1, 224, 224, 3)
    img = preprocess_input(img)
    features = model.predict(img)
    return features

In [5]:
features_img_dict = read_pickle(path=os.path.join(base_path, features_path, model_path, dim_path, f"{model_path}_{dim_path}_img_dict.pkl"))
if features_img_dict is None:  
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    features_img_dict = {k: image_feature(img=v, model=model)[0]
                    for k, v in filtered_img_dict.items()}

    write_pickle(path=os.path.join(base_path, features_path, model_path, dim_path, f"{model_path}_{dim_path}_img_dict.pkl"), obj=features_img_dict)
    print(f"Count of TILE features:   {len(features_img_dict.items())}")
else:
    print("Read from Disk")

path did not exist: /mnt/DataRAID/melismail/PDAC/data/Features_extraction/VGG-16/224x224/VGG-16_224x224_img_dict.pkl


2022-09-18 00:03:42.102748: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-18 00:03:42.615153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:65:00.0, compute capability: 7.0
2022-09-18 00:03:45.459121: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-09-18 00:03:45.847434: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-18 00:03:45.848270: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-09-1

Count of TILE features:   33647


In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
feature_df = pd.DataFrame(data=[{"Pseudonym": k[0], "tile_id": k[1], "Features": v} for k, v in features_img_dict.items()])
write_pickle(path=os.path.join(base_path, features_path, model_path, dim_path, f"{model_path}_{dim_path}_img_df.pkl"), obj=feature_df)